In [1]:
import pandas
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [7]:
df = pickle.load(open("../../pickle/text_attr.pkl", "rb"))
print(df.head())

   title_gme  body_gme  title_hold  body_hold  title_buy  body_buy  \
0          0         1           0          0          0         1   
1          1         0           1          0          0         0   
2          0         0           0          0          0         0   
3          0         1           0          0          0         0   
4          0         1           0          0          0         1   

   title_retard  body_retard  title_wife  body_wife  ...  link  reddit_link  \
0             0            0           0          0  ...     1            1   
1             0            0           0          0  ...     1            0   
2             0            0           0          0  ...     1            1   
3             0            0           0          0  ...     1            1   
4             0            0           0          0  ...     1            1   

   yt_link  tweet_link  facebook_link  gif  score  sin_time  cos_time  day  
0        0           0     

# Wycinanie atrybutów po korelacji

In [8]:
reduced_attr_count = 100
corr_matrix = df.corr(method='spearman')

In [9]:
correlations = []
for i,column in enumerate(corr_matrix):
  x = corr_matrix[column][i+1:]
  for row,v in x.items():
    correlations.append([v,column,row])
correlations.sort(key = lambda x: -abs(x[0]))

In [10]:
score_corr = corr_matrix['score'][corr_matrix['score']<1]
for corr in correlations:
  if len(df.columns) <= reduced_attr_count: break
  c,a1,a2 = corr
  if 'score' in [a1,a2]: continue
  if a1 in df and a2 in df:
    to_be_removed = a1 if score_corr[a1] < score_corr[a2] else a2
    del df[to_be_removed]

# Finałowe wycinanie atrybutów po feature importance

In [11]:
final_attr_count = 30
rf = RandomForestRegressor()
rf.fit(df[df.columns.difference(['score'])], df['score'])

result = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), df.head()), 
             reverse=True)
result

[(0.5435, 'body_moon'),
 (0.0653, 'title_🧻'),
 (0.0307, 'title_guy'),
 (0.0244, 'title_good'),
 (0.0147, 'yt_link'),
 (0.0138, 'tweet_link'),
 (0.0098, 'title_nokia'),
 (0.0088, 'title_stop'),
 (0.0076, 'title_week'),
 (0.0076, 'title_open'),
 (0.0076, 'title_help'),
 (0.0074, 'title_say'),
 (0.0073, 'title_will'),
 (0.0065, 'title_win'),
 (0.0065, 'title_wife'),
 (0.0064, 'body_broker'),
 (0.0062, 'title_shit'),
 (0.006, 'title_right'),
 (0.006, 'title_people'),
 (0.006, 'title_melvin'),
 (0.006, 'title_hold'),
 (0.0059, 'title_not'),
 (0.0058, 'title_trade'),
 (0.0057, 'title_get'),
 (0.0055, 'title_work'),
 (0.0054, 'title_think'),
 (0.0054, 'title_app'),
 (0.005, 'title_strong'),
 (0.0049, 'title_block'),
 (0.0047, 'title_dip'),
 (0.0045, 'title_wsb'),
 (0.0044, 'title_way'),
 (0.0044, 'title_allow'),
 (0.0043, 'title_day'),
 (0.0043, 'body_hedgies'),
 (0.0042, 'title_support'),
 (0.0042, 'title_start'),
 (0.0041, 'title_invest'),
 (0.0039, 'title_fucking'),
 (0.0037, 'title_sell')

In [13]:
selected = []
for x in result[:final_attr_count]:
    selected.append(x[1])
selected_attributes = df[selected + ['score']]
print(selected_attributes)

       body_moon  title_🧻  title_guy  title_good  yt_link  tweet_link  \
0              0        0          0           0        0           0   
1              0        0          0           0        0           0   
2              0        0          0           0        0           0   
3              0        0          0           0        0           0   
4              0        0          0           0        0           0   
...          ...      ...        ...         ...      ...         ...   
38903          0        0          0           0        0           0   
38904          0        0          0           0        0           0   
38905          0        0          0           0        0           0   
38906          0        0          0           0        0           0   
38907          0        0          0           0        0           1   

       title_nokia  title_stop  title_week  title_open  ...  title_not  \
0                0           0           0       

In [14]:
pickle.dump(selected_attributes, open( "../../pickle/text_attributes_processed.pkl", "wb" ))